# Chapter 11: Funding your Account

In [ ]:
# When sending a domestic bank wire, you will need to provide the recipient’s 
# name, address, bank account number, and ABA number (routing number).

# When sending an international bank wire, you will need to provide the recipient’s 
# name, address, banks SWIFT BIC, and bank account number, 
# plus the International Payments System Routing Code, for certain countries (you will be prompted for this).

In [ ]:
pip install plaid-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 518 kB 4.4 MB/s 
     |████████████████████████████████| 140 kB 41.7 MB/s 
  Created wheel for plaid-python: filename=plaid_python-11.2.0-py3-none-any.whl size=2728437 sha256=7f70816a0db7a2a851fc70f5535b1edcb6cb521af52e78d00d284a3a947335ef
  Stored in directory: /root/.cache/pip/wheels/b7/62/6c/7d2b98598a3099f1943d4a01baf2a398d8972454c39d5798f1
Successfully built plaid-python
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


In [ ]:
import plaid
from plaid.api import plaid_api

# Available environments are
# 'Production'
# 'Development'
# 'Sandbox'
configuration = plaid.Configuration(
    host=plaid.Environment.Sandbox,
    api_key={
        'clientId': "6379ff4660579b0013a77c14",
        'secret': "95a8f8027645a1228015b1f6c701d7",
    }
)

api_client = plaid.ApiClient(configuration)
client = plaid_api.PlaidApi(api_client)

In [ ]:
from plaid.model.country_code import CountryCode
from plaid.model.link_token_create_request import LinkTokenCreateRequest
from plaid.model.link_token_create_request_user import LinkTokenCreateRequestUser
from plaid.model.products import Products
import time

CLIENT_NAME = 'Plaid Test'

request = LinkTokenCreateRequest(
        products=[Products('auth'), Products('transactions')],
        client_name=CLIENT_NAME,
        country_codes=[CountryCode('GB')],
        language='en',
        user=LinkTokenCreateRequestUser(
            client_user_id=str(time.time())
        )
    )
# create link token
response = client.link_token_create(request)

# assert on response
link_token = response['link_token']

In [ ]:
from plaid.model.sandbox_public_token_create_request import SandboxPublicTokenCreateRequest
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest
from plaid.model.products import Products

SANDBOX_INSTITUTION = 'ins_109508'

pt_request = SandboxPublicTokenCreateRequest(
        institution_id=SANDBOX_INSTITUTION,
        initial_products=[Products('auth')]
    )

pt_response = client.sandbox_public_token_create(pt_request)

exchange_request = ItemPublicTokenExchangeRequest(
    public_token=pt_response['public_token']
)

In [ ]:
import plaid
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest

# the public token is received from Plaid Link
exchange_request = ItemPublicTokenExchangeRequest(
    public_token=pt_response['public_token']
)
exchange_response = client.item_public_token_exchange(exchange_request)
access_token = exchange_response['access_token']

In [ ]:
from plaid.model.account_type import AccountType
from plaid.model.accounts_get_request import AccountsGetRequest

accounts_request = AccountsGetRequest(
    access_token=access_token
)
accounts_response = client.accounts_get(accounts_request)

account = next(
    acct for acct in accounts_response['accounts'] if acct['type'] == AccountType('depository'))

account_id = account['account_id']

In [ ]:
account_id

'aqyaKraXmeCLvGbgaZg4IW4nEqAMjytP5pA5l'

In [ ]:
from plaid.model.bank_transfer_create_request import BankTransferCreateRequest
from plaid.model.bank_transfer_network import BankTransferNetwork
from plaid.model.bank_transfer_idempotency_key import BankTransferIdempotencyKey
from plaid.model.bank_transfer_type import BankTransferType
from plaid.model.bank_transfer_user import BankTransferUser
from plaid.model.ach_class import ACHClass
from random import random

bt_request = BankTransferCreateRequest(
        idempotency_key=BankTransferIdempotencyKey(str(random())),
        access_token=access_token,
        account_id=account_id,
        type=BankTransferType('credit'),
        network=BankTransferNetwork('ach'),
        amount='1.00',
        iso_currency_code='USD',
        description='test',
        user=BankTransferUser(legal_name='Firstname Lastname'),
        ach_class=ACHClass('ppd'),
        custom_tag='',
    )

bt_response = client.bank_transfer_create(bt_request)

ApiException: ignored